# Quickstart

To get started with LSDB we will demonstrate a very common workflow. It consists of crossmatching a small set of objects of your interest with a large survey catalog stored in HiPSCat format (Gaia), applying cuts to the data, and saving the final result.

The first thing you need to do is to import our package.

In [ ]:
import lsdb

Create a Pandas Dataframe with the equatorial coordinates (right ascension and declination, in degrees) for your objects.

In [ ]:
import pandas as pd

# The coordinates (ra, dec) for our objects of interest
objects = [(180.080612, 9.507394), (179.884664, 10.479632), (179.790319, 9.551745)]

objects_df = pd.DataFrame(objects, columns=["ra", "dec"])
objects_df

Now that the data is in a DataFrame you can create an LSDB catalog to have it in HiPSCat format.

In [ ]:
my_object_catalog = lsdb.from_dataframe(objects_df, catalog_name="my_object_catalog", catalog_type="object")
my_object_catalog

Next, read the catalog we want to crossmatch with. Because we are downloading it from a web source we need to install an additional package (__aiohttp__). If your catalog happens to be present in local storage you can call `read_hipscat` directly.

In [ ]:
!pip install aiohttp --quiet

In this tutorial we will read a small 1 degree cone region of Gaia DR3, one that should contain our objects. While LSDB typically reads into memory only the minimal amount of data it needs for our workflow, manually providing it with spatial information helps it identify which files to search for on disk.

In [ ]:
from lsdb.core.search import ConeSearch

gaia_path = "https://epyc.astro.washington.edu/~lincc-frameworks/hipscat_surveys/gaia_dr3/gaia"

gaia = lsdb.read_hipscat(gaia_path, search_filter=ConeSearch(ra=180, dec=10, radius_arcsec=0.5 * 3600))
gaia

Let's crossmatch them! As a result we will have a catalog with the objects from Gaia that match our initial objects of interest, according to a specified maximum distance. We will use the default K nearest neighbors algorithm with `k=1` and a maximum separation distance of 1 arcsecond.

In [ ]:
result = my_object_catalog.crossmatch(gaia, n_neighbors=1, radius_arcsec=1 * 3600, require_right_margin=False)

Now let's say we wish to apply a cut to our data to get all the objects with a mean magnitude in the G band greater than 18 and a total number of observations AL greater than 200. We can build a query expression and filter the catalog. Because of the crossmatch, the name of the Gaia columns for the query need to contain the name of the catalog as a suffix, `_gaia`.

In [ ]:
result = result.query("phot_g_mean_mag_gaia > 18 and astrometric_n_obs_al_gaia > 200")

Our workflow takes advantage of Dask's "lazy" evaluation. This means that we have been defining a set of tasks which will only be executed at our command. When that happens, data will be read into disk and operations will be distributed among the several workers for parallel computation. To trigger this we will call `compute` on the catalog that resulted from the crossmatch. The final result will be presented in a Pandas DataFrame.

In [ ]:
result.compute()

You will want to save your resulting catalog to disk, especially if it is too large to fit in memory or if you will need to use it later on. The catalog exposes the `to_hipscat` API for that, you just need to provide it with a path for the target `base directory` and a `catalog name`.

In [ ]:
result.to_hipscat("lsdb_catalogs", catalog_name="my_object_catalog_x_gaia")